In [1]:
#export
from typing import List, Iterator, Any, NewType, TypeVar, Generic
import k1lib.cli as cli; from numbers import Number
import k1lib, itertools, copy, torch; import numpy as np

__all__ = ["BaseCli", "Table", "T", "fastF",
           "serial", "oneToMany", "manyToMany", "mtmS"]

In [2]:
#export
settings = k1lib.Settings()
atomic = k1lib.Settings()
settings.add("jit", True, "whether to enable automatic JIT compilation of cli tools. See `fastF` for more details")
settings.add("atomic", atomic, "classes/types that are considered atomic and specified cli tools should never try to iterate over them")
settings.add("defaultDelim", "\t", "default delimiter used in-between columns when creating tables. Defaulted to tab character.")
settings.add("defaultIndent", "  ", "default indent used for displaying nested structures")
settings.add("strict", False, "turning it on can help you debug stuff, but could also be a pain to work with")
settings.add("inf", float("inf"), "infinity definition for many clis. Here because you might want to temporarily not loop things infinitely")
k1lib.settings.add("cli", settings, "from k1lib.cli module")

Settings:                                                   
- displayCutoff = 50                                        ​cutoff length when displaying a Settings object                                                                     
- svgScale      = 0.7                                       ​default svg scales for clis that displays graphviz graphs                                                           
- wd            = /home/kelvin/repos/labs/k1lib/k1lib/cli   ​default working directory, will get from `os.getcwd()`. Will update using `os.chdir()` automatically when changed   
- cli           = <Settings>                                ​from k1lib.cli module                                                                                               
  - jit           = True                                    ​whether to enable automatic JIT compilation of cli tools. See `fastF` for more details                              
  - defaultDelim  = 	                            

In [3]:
#export
def patchDefaultDelim(st:str):
    """
:param s:
    - if not None, returns self
    - else returns the default delimiter in :attr:`~k1lib.settings`"""
    return settings.defaultDelim if st is None else st
def patchDefaultIndent(st:str):
    """
:param s:
    - if not None, returns self
    - else returns the default indent character in :attr:`~k1lib.settings`"""
    return settings.defaultIndent if st is None else st

In [4]:
#export
T = TypeVar("T")
"""Generic type variable"""
class _MetaType(type):
    def __getitem__(self, generic):
        d = {"__args__": generic, "_n": self._n, "__doc__": self.__doc__}
        return _MetaType(self._n, (), d)
    def __repr__(self):
        def main(self):
            def trueName(o):
                if isinstance(o, _MetaType): return main(o)
                try: return o.__name__
                except: return f"{o}"
            if hasattr(self, "__args__"):
                if isinstance(self.__args__, tuple):
                    return f"{self._n}[{', '.join([trueName(e) for e in self.__args__])}]"
                else: return f"{self._n}[{trueName(self.__args__)}]"
            return self._n
        return main(self)
def newTypeHint(name, docs=""):
    """Creates a new type hint that can be sliced and yet still looks fine
in sphinx. Crudely written by my poorly understood idea of Python's
metaclasses. Seriously, this shit is bonkers, read over it https://stackoverflow.com/questions/100003/what-are-metaclasses-in-python

Example::

    Table = newTypeHint("Table", "some docs")
    Table[int] # prints out as "Table[int]", and sphinx fell for it too
    Table[Table[str], float] # prints out as "Table[Table[str], float]"
"""
    return _MetaType(name, (), {"_n": name, "__doc__": docs})
#Table = newTypeHint("Table", """Essentially just Iterator[List[T]]. This class is just here so that I can generate the docs with nicely formatted types like "Table[str]".""")
#Table = NewType("Table", List)
class Table(Generic[T]):
    """Essentially just Iterator[List[T]]. This class is just here so that I can generate the docs with nicely formatted types like "Table[str]"."""
    pass
Table._name = "Table"
#Table.__module__ = "cli"
class Row(list):
    """Not really used currently. Just here for potential future feature"""
    pass

In [5]:
#export
class BaseCli:
    """A base class for all the cli stuff. You can definitely create new cli tools that
have the same feel without extending from this class, but advanced stream operations
(like ``+``, ``&``, ``.all()``, ``|``) won't work.

At the moment, you don't have to call super().__init__() and super().__ror__(),
as __init__'s only job right now is to solidify any :class:`~k1lib.cli.modifier.op`
passed to it, and __ror__ does nothing."""
    def __init__(self, fs:list=[]):
        """Not expected to be instantiated by the end user.

**fs param**

Expected to use it like this::

    class A(BaseCli):
        def __init__(self, f):
            fs = [f]; super().__init__(fs); self.f = fs[0]

Where ``f`` is some (potentially exotic) function. This will replace f with a "normal"
function that's executable. See source code of :class:`~k1lib.cli.filt.filt` for an
example of why this is useful. Currently, it will:

- Replace with last recorded ``4 in op()``, if ``f`` is :data:`True`, because Python does
  not allow returning complex objects from __contains__ method
- Solidifies every :class:`~k1lib.cli.modifier.op`."""
        if isinstance(fs, tuple): raise AttributeError("`fs` should not be a tuple. Use a list instead, so that new functions can be returned")
        l = []
        for f in fs:
            if f is True: f = cli.op._op_pop_contains()
            if f is False: f = cli.op._op_pop_contains_inv()
            if f.__class__.__name__.split(".")[-1] == "op":
                f.op_solidify()
            l.append(f)
        fs.clear(); fs.extend(l)
    def __and__(self, cli:"BaseCli") -> "oneToMany":
        """Duplicates input stream to multiple joined clis.
Example::

    # returns [[5], [0, 1, 2, 3, 4]]
    range(5) | (shape() & iden()) | deref()

Kinda like :class:`~k1lib.cli.modifier.apply`. There're just multiple ways of doing
this. This I think, is more intuitive, and :class:`~k1lib.cli.modifier.apply` is more
for lambdas and columns mode. Performances are pretty much identical."""
        if isinstance(self, oneToMany): return self._after(cli)
        if isinstance(cli, oneToMany): return cli._before(self)
        return oneToMany(self, cli)
    def __add__(self, cli:"BaseCli") -> "mtmS":
        """Parallel pass multiple streams to multiple clis.
Example::

    # returns [8, 15]
    [2, 3] | ((op() * 4) + (op() * 5)) | deref()"""
        if isinstance(self, mtmS): return self._after(cli)
        if isinstance(cli, mtmS): return cli._before(self)
        return mtmS(self, cli)
    def all(self, n:int=1) -> "BaseCli":
        """Applies this cli to all incoming streams.
Example::

    # returns (3,)
    torch.randn(3, 4) | toMean().all() | shape()
    # returns (3, 4)
    torch.randn(3, 4, 5) | toMean().all(2) | shape()

:param n: how many times should I chain ``.all()``?"""
        if n < 0: raise AttributeError(f"Does not make sense for `n` to be \"{n}\"")
        s = self
        for i in range(n): s = manyToMany(s)
        return s
    def __or__(self, cli) -> "serial":
        """Joins clis end-to-end.
Example::

    c = apply(op() ** 2) | deref()
    # returns [0, 1, 4, 9, 16]
    range(5) | c"""
        if isinstance(self, serial): return self._after(cli)
        if isinstance(cli, serial): return cli._before(self)
        return serial(self, cli)
    def __ror__(self, it): return NotImplemented
    def f(self) -> Table[Table[int]]:
        """Creates a normal function :math:`f(x)` which is equivalent to
``x | self``."""
        return lambda it: self.__ror__(it)
    def __lt__(self, it):
        """Backup pipe symbol `>`, purely for style, so that you can do something like
this::

    range(4) > file("a.txt")"""
        return self.__ror__(it)
    def __call__(self, it, *args):
        """Another way to do ``it | cli``. If multiple arguments are fed, then the
argument list is passed to cli instead of just the first element. Example::

    @applyS
    def f(it):
        return it
    f(2) # returns 2
    f(2, 3) # returns [2, 3]"""
        if len(args) == 0: return self.__ror__(it)
        else: return self.__ror__([it, *args])
    def __neg__(self):
        """Alias for __invert__, for clis that support inverting stuff."""
        return ~self

In [6]:
assert [2, 3] | ((cli.op() * 4) + (cli.op() * 5)) | cli.deref() == [8, 15]
assert torch.randn(3, 4) | cli.toMean().all() | cli.shape() == (3,)
assert torch.randn(3, 4, 5) | cli.toMean().all(2) | cli.shape() == (3, 4)
c = cli.apply(cli.op() ** 2) | cli.deref()
assert range(5) | c == [0, 1, 4, 9, 16]
assert (range(4) > cli.file("test/init.txt")) == "test/init.txt"

In [7]:
#export
def fastF(c, x=None):
    """Tries to figure out what's going on, is it a normal function, or an applyS,
or a BaseCli, etc., and return a really fast function for execution. Example::

    # both returns 16, fastF returns "lambda x: x**2", so it's really fast
    fastF(op()**2)(4)
    fastF(applyS(lambda x: x**2))(4)

At the moment, parameter ``x`` does nothing, but potentially in the future, you can
pass in an example input to the cli, so that this returns an optimized, C compiled
version.

:param x: sample data for the cli"""
    if isinstance(c, cli.op): return c.ab_fastF()
    if isinstance(c, cli.applyS): return fastF(c.f)
    if isinstance(c, BaseCli): return c.__ror__
    return c

In [8]:
assert fastF(cli.op() ** 2)(4) == 16

In [9]:
#export
class serial(BaseCli):
    def __init__(self, *clis:List[BaseCli]):
        """Merges clis into 1, feeding end to end. Used in chaining clis
together without a prime iterator. Meaning, without this, stuff like this
fails to run::

    [1, 2] | a() | b() # runs
    c = a() | b(); [1, 2] | c # doesn't run if this class doesn't exist"""
        fs = list(clis); super().__init__(fs); self.clis = fs; self._cache()
    def _cache(self):
        self._hasTrace = any(isinstance(c, cli.trace) for c in self.clis)
        self._cliCs = [fastF(c) for c in self.clis]; return self
    def __ror__(self, it:Iterator[Any]) -> Iterator[Any]:
        if self._hasTrace: # slower, but tracable
            for cli in self.clis: it = it | cli
        else: # faster, but not tracable
            for cli in self._cliCs: it = cli(it)
        return it
    def _before(self, c): self.clis = [c] + self.clis; return self._cache()
    def _after(self, c): self.clis = self.clis + [c]; return self._cache()

In [10]:
@cli.applyS
def f(it): return it
assert f(2) == 2
assert f(2, 3) == [2, 3]
assert range(5) | (cli.shape() & cli.iden()) | cli.deref() == [[5], [0, 1, 2, 3, 4]]
assert isinstance([np.random.randn(2, 3, 4)] | (cli.item() | serial(cli.shape())), tuple)

In [11]:
# examples in cli rst docs
assert cli.shape()(np.random.randn(2, 3, 5)) == (2, 3, 5)
assert [np.random.randn(2, 3, 5)] | (cli.item() | cli.shape()) == (2, 3, 5)

In [12]:
#export
atomic.add("baseAnd", (Number, np.number, str, dict, bool, bytes, list, tuple, torch.Tensor), "used by BaseCli.__and__")
def _iterable(it):
    try: iter(it); return True
    except: return False
class oneToMany(BaseCli):
    def __init__(self, *clis:List[BaseCli]):
        """Duplicates 1 stream into multiple streams, each for a cli in the
list. Used in the "a & b" joining operator. See also: :meth:`BaseCli.__and__`"""
        fs = list(clis); super().__init__(fs); self.clis = fs; self._cache()
    def __ror__(self, it:Iterator[Any]) -> Iterator[Iterator[Any]]:
        if isinstance(it, atomic.baseAnd) or not _iterable(it):
            for cli in self._cliCs: yield cli(it)
        else:
            its = itertools.tee(it, len(self.clis))
            for cli, it in zip(self._cliCs, its): yield cli(it)
    def _cache(self): self._cliCs = [fastF(c) for c in self.clis]; return self
    def _before(self, c): self.clis = [c] + self.clis; return self._cache()
    def _after(self, c): self.clis = self.clis + [c]; return self._cache()

In [13]:
#export
class manyToMany(BaseCli):
    def __init__(self, cli:BaseCli):
        """Applies multiple streams to a single cli. Used in the :meth:`BaseCli.all`
operator."""
        fs = [cli]; super().__init__(fs); self.cli = fs[0]; self._cliC = fastF(self.cli)
    def __ror__(self, it:Iterator[Iterator[Any]]) -> Iterator[Iterator[Any]]:
        f = self._cliC; return (f(s) for s in it)

In [14]:
#export
class mtmS(BaseCli):
    def __init__(self, *clis:List[BaseCli]):
        """Applies multiple streams to multiple clis independently. Used in
the "a + b" joining operator. See also: :meth:`BaseCli.__add__`.

Weird name is actually a shorthand for "many to many specific"."""
        fs = list(clis); super().__init__(fs=fs); self.clis = fs; self._cache()
    def _cache(self): self._cliCs = [fastF(c) for c in self.clis]; return self
    def _before(self, c): self.clis = [c] + self.clis; return self._cache()
    def _after(self, c): self.clis = self.clis + [c]; return self._cache()
    def __ror__(self, its:Iterator[Any]) -> Iterator[Any]:
        for cli, it in zip(self._cliCs, its): yield cli(it)
    @staticmethod
    def f(f, i:int, n:int=100):
        """Convenience method, so
that this::

    mtmS(iden(), op()**2, iden(), iden(), iden())
    # also the same as this btw:
    (iden() + op()**2 + iden() + iden() + iden())

is the same as this::

    mtmS.single(op()**2, 1, 5)

Example::

    # returns [5, 36, 7, 8, 9]
    range(5, 10) | mtmS.single(op()**2, 1, 5) | deref()

:param i: where should I put the function?
:param n: how many clis in total? Defaulted to 100"""
        return mtmS(*([cli.iden()]*i + [f] + [cli.iden()]*(n-i-1)))

In [15]:
assert len((cli.iden() + cli.op()**2 + cli.iden() + cli.iden() + cli.iden()).clis) == 5
assert len(mtmS(cli.iden(), cli.op()**2, cli.iden(), cli.iden(), cli.iden()).clis) == 5
assert range(5, 10) | mtmS.f(cli.op()**2, 1, 5) | cli.deref() == [5, 36, 7, 8, 9]

In [17]:
!../../export.py cli/init

Current dir: /home/kelvin/repos/labs/k1lib, ../../export.py
rm: cannot remove 'build': No such file or directory
rm: cannot remove '__pycache__': No such file or directory
Found existing installation: k1lib 0.13
Uninstalling k1lib-0.13:
  Successfully uninstalled k1lib-0.13
running install
/home/kelvin/anaconda3/envs/torch/lib/python3.8/site-packages/setuptools/command/install.py:34: SetuptoolsDeprecationWarning: setup.py install is deprecated. Use build and pip and other standards-based tools.
  warnings.warn(
/home/kelvin/anaconda3/envs/torch/lib/python3.8/site-packages/setuptools/command/easy_install.py:156: EasyInstallDeprecationWarning: easy_install command is deprecated. Use build and pip and other standards-based tools.
  warnings.warn(
running bdist_egg
running egg_info
creating k1lib.egg-info
writing k1lib.egg-info/PKG-INFO
writing dependency_links to k1lib.egg-info/dependency_links.txt
writing requirements to k1lib.egg-info/requires.txt
writing top-level names to k1lib.egg-in